## This is my attempt to use siamese with gan idea

In [1]:
use_kaggle = 0

if use_kaggle:
    #!pip install git+https://github.com/fastai/fastai.git
    !git clone https://github.com/benwu232/humpback
    import sys
     # Add directory holding utility functions to path to allow importing utility funcitons
    #sys.path.insert(0, '/kaggle/working/protein-atlas-fastai')
    sys.path.append('/kaggle/humback/')

In [2]:
# Suppress annoying stderr output when importing keras.
import sys
import platform
old_stderr = sys.stderr
sys.stderr = open('/dev/null' if platform.system() != 'Windows' else 'nul', 'w')

sys.stderr = old_stderr

import random
from scipy.ndimage import affine_transform

import pickle
import numpy as np
from math import sqrt

# Determise the size of each image
from os.path import isfile
from PIL import Image as pil_image
from tqdm import tqdm_notebook

from pandas import read_csv
import pandas as pd
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy_thresh
from fastai.basic_data import *
from torch.utils.data import DataLoader, Dataset
from torch import nn
from fastai.callbacks.hooks import num_features_model, model_sizes
from fastai.layers import BCEWithLogitsFlat
from fastai.basic_train import Learner
from skimage.util import montage
import pandas as pd
from torch import optim
import re

if use_kaggle:
    from humpback.utils import *
else:
    from utils import *
    
from IPython.core.debugger import set_trace
#from functional import seq
%load_ext autoreload
%autoreload 2

In [3]:
fastai.__version__

'1.0.44.dev0'

In [4]:
%connect_info

{
  "shell_port": 47332,
  "iopub_port": 33229,
  "stdin_port": 60831,
  "control_port": 40941,
  "hb_port": 43430,
  "ip": "127.0.0.1",
  "key": "12129e0f-da88447a9c8647dfdce78cf0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-0cf0cb30-675b-418d-9c43-3e93783a221a.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [5]:
root_path = Path('../input')
train_path = root_path/'train'
test_path = root_path/'test'
learn_path = Path('../')

USE_CUDA = torch.cuda.is_available()

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if USE_CUDA else "cpu")

In [6]:
name = f'siamese_resnet18_224'

#arch = models.resnet50
arch = models.resnet18
im_size = 224
train_batch_size = 64
val_batch_size = 128
if use_kaggle:
    dl_workers = 0
else:
    dl_workers = 6
SEED=0
emb_len = 256

In [7]:
df0 = pd.read_csv('../input/train.csv')
df_new = df0[df0.Id == 'new_whale']
df_known = df0[df0.Id != 'new_whale']
train_list, val_list = split_whale_set(df0, nth_fold=0, new_whale_method=1, seed=1)

im_count = df0[df0.Id != 'new_whale'].Id.value_counts()
im_count.name = 'sighting_count'
ex_df = df0.join(im_count, on='Id')

path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)
fn2label = {row[1].Image: row[1].Id for row in df0.iterrows()}
class_dict = make_whale_class_dict(df0)
file_lut = df0.set_index('Image').to_dict()

In [8]:
len(val_list)

5471

In [9]:
im_tfms = get_transforms(do_flip=False, max_zoom=1, max_warp=0, max_rotate=2)

data = (
    ImageItemList
        # .from_df(df_known, 'data/train', cols=['Image'])
        .from_folder(train_path)
        # .split_by_idxs(train_item_list, val_item_list)
        .split_by_valid_func(lambda path: path2fn(str(path)) in val_list)
        # .split_by_idx(val_list)
        # .random_split_by_pct(seed=SEED)
        .label_from_func(lambda path: fn2label[path2fn(str(path))])
        #.add_test(ImageItemList.from_folder(test_path))
        #.transform([None, None], size=im_size, resize_method=ResizeMethod.SQUISH)
        .transform(im_tfms, size=im_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=train_batch_size, num_workers=dl_workers, path=root_path)
        #.normalize(imagenet_stats)
)

data.add_tfm(normalize_batch)

train_dl = DataLoader(
    SiameseDs(data.train_ds, data.train_dl),
    batch_size=train_batch_size,
    shuffle=True,
    collate_fn=collate_siamese,
    num_workers=dl_workers
)

In [10]:
data_v = (
    ImageItemList
        # .from_df(df_known, 'data/train', cols=['Image'])
        .from_folder(train_path)
        # .split_by_idxs(train_item_list, val_item_list)
        .split_by_valid_func(lambda path: path2fn(str(path)) in val_list)
        # .split_by_idx(val_list)
        # .random_split_by_pct(seed=SEED)
        .label_from_func(lambda path: fn2label[path2fn(str(path))])
        .add_test(ImageItemList.from_folder(test_path))
        #.transform([None, None], size=im_size, resize_method=ResizeMethod.SQUISH)
        #.transform(im_tfms, size=im_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=train_batch_size, num_workers=dl_workers, path=root_path)
        .normalize(imagenet_stats)
)

#v = SimpleDataset(data.valid)
valid_dl = DataLoader(
    SimpleDataset(data_v.valid_ds),
    batch_size=val_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=dl_workers
)

test_dl = DataLoader(
    SimpleDataset(data_v.test_ds),
    batch_size=val_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=dl_workers
)

data_ref = (
    ImageItemList
        .from_df(df_known, train_path, cols=['Image'])
        #.from_folder(train_path)
        # .split_by_idxs(train_item_list, val_item_list)
        .split_by_valid_func(lambda path: path2fn(str(path)) in val_list)
        # .split_by_idx(val_list)
        # .random_split_by_pct(seed=SEED)
        .label_from_func(lambda path: fn2label[path2fn(str(path))])
        #.add_test(ImageItemList.from_folder(test_path))
        #.transform([None, None], size=im_size, resize_method=ResizeMethod.SQUISH)
        #.transform(im_tfms, size=im_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=val_batch_size, num_workers=dl_workers, path=root_path)
        .normalize(imagenet_stats)
)
print(len(data_ref.train_ds), len(data_ref.valid_ds))

ref_dl = DataLoader(
        SimpleDataset(data_ref.train_ds),
        batch_size=val_batch_size,
        shuffle=False,
        #collate_fn=siamese_collate,
        num_workers=dl_workers
    )

/media/wb/T08/study/fastai/fastai/basic_data.py:243: UserWarning: It's not possible to collate samples of your dataset together in a batch.
Shapes of the inputs/targets:
[[torch.Size([3, 326, 572]), torch.Size([3, 788, 1050]), torch.Size([3, 179, 312]), torch.Size([3, 343, 640]), torch.Size([3, 441, 1050]), torch.Size([3, 445, 778]), torch.Size([3, 407, 700]), torch.Size([3, 684, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 191, 1050]), torch.Size([3, 466, 815]), torch.Size([3, 342, 513]), torch.Size([3, 337, 1050]), torch.Size([3, 450, 1050]), torch.Size([3, 386, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 259, 453]), torch.Size([3, 678, 1017]), torch.Size([3, 600, 1050]), torch.Size([3, 525, 1050]), torch.Size([3, 220, 366]), torch.Size([3, 525, 1050]), torch.Size([3, 450, 1050]), torch.Size([3, 425, 705]), torch.Size([3, 600, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 415, 1050]), torch.Size([3, 251, 640]), torch.Size([3, 466, 1050]),

You can deactivate this warning by passing `no_check=True`.
You can deactivate this warning by passing `no_check=True`.
12158 3539


/media/wb/T08/study/fastai/fastai/basic_data.py:243: UserWarning: It's not possible to collate samples of your dataset together in a batch.
Shapes of the inputs/targets:
[[torch.Size([3, 699, 1050]), torch.Size([3, 498, 873]), torch.Size([3, 635, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 773, 1050]), torch.Size([3, 352, 1050]), torch.Size([3, 674, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 380, 1050]), torch.Size([3, 714, 1050]), torch.Size([3, 306, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 525, 1050]), torch.Size([3, 477, 1050]), torch.Size([3, 469, 1020]), torch.Size([3, 242, 1050]), torch.Size([3, 387, 1050]), torch.Size([3, 625, 937]), torch.Size([3, 429, 1050]), torch.Size([3, 725, 1016]), torch.Size([3, 547, 957]), torch.Size([3, 325, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 591, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 700, 1050]), torch.Size([3, 600, 1050]), torch.Size([3, 415,

In [11]:
data_bunch = ImageDataBunch(train_dl, valid_dl, fix_dl=ref_dl, device=device)

In [12]:
#siamese = SiameseNetwork(arch=arch)
siamese = SiameseNet(emb_len=emb_len, arch=arch, forward_type='similarity', drop_rate=0.2)
siamese.to(device)

SiameseNet(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (r

In [13]:
learn = LearnerEx(data_bunch,
                  siamese,
                  enable_validate=False,
                  loss_func=BCEWithLogitsFlat(),
                  path=learn_path,
                  #loss_func=contrastive_loss,
                  metrics=[lambda preds, targs: accuracy_thresh(preds.squeeze(), targs, sigmoid=False)]
                  )

In [14]:
learn.split([learn.model.cnn[:6], learn.model.cnn[6:], learn.model.fc])

In [16]:
learn.freeze_to(-1)
learn.fit_one_cycle(1)
learn.unfreeze()

In [15]:
enable_lr_find = 1
if enable_lr_find:
    print('LR plotting ...')
    learn.lr_find()
    learn.recorder.plot()
    plt.savefig('lr_find.png')

LR plotting ...


PicklingError: Can't pickle <function <lambda> at 0x7fd4027c18c8>: attribute lookup <lambda> on __main__ failed

In [15]:
from fastai.callbacks import SaveModelCallback
cb_save_model = SaveModelCallback(learn, every="epoch", name=f"siamese_cl")
#cb_siamese_validate = SiameseValidateCallback(learn, txlog)
cbs = [cb_save_model]#, cb_siamese_validate]

In [ ]:
max_lr = 1e-3
lrs = [max_lr/100, max_lr/10, max_lr]
learn.fit_one_cycle(100, lrs, callbacks=cbs)

For training data, we are taking all the images except for the category - New Whale

In [ ]:
new_df = df[df['Id']!= 'new_whale']
#new_df = new_df[new_df['sighting_count']>1]

print('shape of data for training ',new_df.shape)
new_df.drop(columns = ['sighting_count'] , inplace=True)
new_df.head(2)

In [ ]:
new_df.reset_index(inplace=True)
new_df.drop(columns='index' , inplace=True)
new_df.tail(3)

In [ ]:
new_df.to_records()

In [ ]:
"""
The data set we are using for training contains all images except of new whales.
we don't require creating phase values for this  datasets , as duplicate images are very few.
i am using index present in train.csv as the phase value as we can use it for indexing very easily 
"""

tagged = dict([(p,w) for _,p,w in new_df.to_records()])
h2ps = dict([(idx , p ) for   idx,p,w in new_df.to_records()])
p2h   = dict([(p , idx) for idx , p , w in new_df.to_records()])
h2p = h2ps.copy()
join = tagged.copy()

In [ ]:
def expand_path(p):
    if isfile(train_path/p): 
        return train_path/p
    if isfile(test_path/p): 
        return test_path/p
    return p

p2size = {}
for p in tqdm_notebook(join):
    size      = pil_image.open(expand_path(p)).size
    p2size[p] = size
len(p2size), list(p2size.items())[:5]

In [ ]:
## phase value for all categories except new whale
h2ws = {}
new_whale = 'new_whale'
for p,w in tagged.items():
    if w != new_whale: # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
for h,ws in h2ws.items():
    if len(ws) > 1:
        h2ws[h] = sorted(ws)
len(h2ws)

In [ ]:
## for each whale category, observe the associated phase values , 
##store all whale categories even the categories with just one image ( this is  different from martin's approach)

w2hs = {}
for h,ws in h2ws.items():
    if len(ws) == 1: # Use only unambiguous pictures

        w = ws[0]
        if w not in w2hs: w2hs[w] = []
        if h not in w2hs[w]: w2hs[w].append(h)
for w,hs in w2hs.items():
    #if len(hs) > 1:
        w2hs[w] = sorted(hs)
len(w2hs)

In [ ]:
len(h2ws)

In [ ]:
def read_raw_image(p):
    img = pil_image.open(expand_path(p))
    return img


import matplotlib.pyplot as plt


In [ ]:
train = [] # A list of  indices of images to be used in training data.
for hs in w2hs.values():
    if len(hs) >= 1:
        train += hs
random.shuffle(train)
train_set = set(train)

In [ ]:
## we have whales categories with phases(images) more than 1. shuffle the phase values now.
w2ts = {} #Associate the image index from train to each whale id.
for w,hs in w2hs.items():
    for h in hs:
        if h in train_set:
            if w not in w2ts: w2ts[w] = []
            if h not in w2ts[w]: w2ts[w].append(h)
for w,ts in w2ts.items(): w2ts[w] = np.array(ts)
## then again for each whale categories see how many images you have , 
## you are working with 5004 whale categories and 15697 images 
    
    
t2i = {} # The position in train of each training image id
for i,t in enumerate(train): t2i[t] = i

len(train),len(w2ts)

In [ ]:
# from keras.utils import Sequence
# import keras

from IPython.core.debugger import set_trace
import random
#from keras import backend as K

try:
    from lapjv import lapjv
    segment = False
except ImportError:
    print('Module lap not found, emulating with much slower scipy.optimize.linear_sum_assignment')
    segment = True
    from scipy.optimize import linear_sum_assignment


Import functions from fast ai library

In [ ]:

fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}  #new_
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)


Creating dataset for all the training images. Because of some reason , i am not able to create validation set as well ( produces error while indexing from match and unmatch matrices. If someone is able to find the work arounf the help will be appreciated

In [ ]:
classes = df.Id.unique()
data = (
    ImageItemList  ##df[(df.Id != 'new_whale') & (df.sighting_count >1)]
        .from_df( df[(df.Id != 'new_whale')], train_path, cols=['Image'])
        .no_split()##split_by_valid_func(lambda path: path2fn(path) in val_fns) 
        .label_from_func(lambda path: fn2label[path2fn(path)] ,  classes=classes)
        .add_test(ImageItemList.from_folder(test_path))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH))

In [ ]:
print(len(train))
print(len(data.train.x))
print(len(data.valid.x))

In [ ]:
from IPython.core.debugger import set_trace
import random

# First try to use lapjv Linear Assignment Problem solver as it is much faster.
# At the time I am writing this, kaggle kernel with custom package fail to commit.
# scipy can be used as a fallback, but it is too slow to run this kernel under the time limit
# As a workaround, use scipy with data partitioning.
# Because algorithm is O(n^3), small partitions are much faster, but not what produced the submitted solution
try:
    from lapjv import lapjv
    segment = False
except ImportError:
    print('Module lap not found, emulating with much slower scipy.optimize.linear_sum_assignment')
    segment = True
    from scipy.optimize import linear_sum_assignment


TwoImDataset creation is the part where I am trying to replicate 'TrainingData Class' from https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563/output
For whale categories having just one images in training data , matching pair -  same image pair (A,A) . For other categories it creates a de arrangement.  

In [ ]:
np.random.random()

In [ ]:
import pdb
def is_even(num): return num % 2 == 0

class TwoImDataset(Dataset):
    def __init__(self, ds, score, steps = 1000):
        self.ds = ds
        self.whale_ids = ds.y.items
        self.steps =1000
        self.score  = -score
        for ts in w2ts.values():
            idxs =  ts.copy() #[t2i[t] for t in ts]
            #idxs = [i for i in  idxs if i <score.shape[0]]
            for i in idxs:
                for j in idxs:
                    self.score[i,j] = 10000.0   # Set a large value for matching
        self.epsilon = 1.0
        self.on_epoch_end()
        
    def set_epsilon(self, epsilon=1.0):
        self.epsilon = epsilon
        
    def __len__(self):
        return 2 * len(self.ds)

    def __getitem__(self, idx):
        prob = np.random.random()
        choice = idx % 2 == 0
        if choice:
            idx //=2
        else:
            idx = (idx - 1) // 2
        
        if prob < self.epsilon: 
            return self.sample_random(idx, choice)
        else:
            return self.sample_policy(idx, choice)
                
    def sample_random(self, idx, choice):
        if choice:
            return self.sample_same(idx)
        else: return self.sample_different(idx)

    def sample_same(self, idx):
        whale_id = self.whale_ids[idx]        
        candidates = list(np.where(self.whale_ids == whale_id)[0])
        candidates.remove(idx) # dropping our current whale - we don't want to compare against an identical image!
        
        if len(candidates) == 0: # oops, there is only a single whale with this id in the dataset
            return self.sample_different(idx)
        
        np.random.shuffle(candidates)
        return self.construct_example(self.ds[idx][0], self.ds[candidates[0]][0], 1)
    
    def sample_different(self, idx):
        whale_id = self.whale_ids[idx]
        candidates = list(np.where(self.whale_ids != whale_id)[0])
        np.random.shuffle(candidates)
        return self.construct_example(self.ds[idx][0], self.ds[candidates[0]][0], 0)
  
    def sample_policy(self, idx , tag):
        #set_trace()
        if tag==0:
            first_image_id =  self.match[idx][0]
            second_image_id = self.match[idx][1]
            #if first_image_id < len(self.ds) and second_image_id< len(self.ds):         
            return self.construct_example(self.ds[first_image_id][0], self.ds[second_image_id][0], 1)
        else:
            first_image_id =  self.unmatch[idx][0]
            second_image_id = self.unmatch[idx][1]     
            return self.construct_example(self.ds[first_image_id][0], self.ds[second_image_id][0], 0)
  
    def on_epoch_end(self):
        if self.steps <= 0: return # Skip this on the last epoch.
        self.steps     -= 1
        self.match      = []
        self.unmatch    = []
        if segment:
            tmp   = []
            batch = 512
            for start in range(0, score.shape[0], batch):
                end = min(score.shape[0], start + batch)
                _, x = linear_sum_assignment(self.score[start:end, start:end])
                tmp.append(x + start)
            x = np.concatenate(tmp)
        else:
            #print('using lapjv')
            x,_,_ = lapjv(self.score) # Solve the linear assignment problem
        y = np.arange(len(x),dtype=np.int32)

        # Compute a derangement for matching whales
        for ts in w2ts.values():
            d = ts.copy()
            if (len(d)==1):
                for ab in zip(ts,d): self.match.append(ab)
            else:                
                while True:
                    random.shuffle(d)
                    if not np.any(ts == d): break
                for ab in zip(ts,d): self.match.append(ab)

        # Construct unmatched whale pairs from the LAP solution.
        for i,j in zip(x,y):
            if i == j:
                print(self.score)
                print(x)
                print(y)
                print(i,j)
            assert i != j
            self.unmatch.append((train[i],train[j]))

        # Force a different choice for an eventual next epoch.
        self.score[x,y] = 10000.0
        self.score[y,x] = 10000.0
        random.shuffle(self.match)
        random.shuffle(self.unmatch)
        print('end of epoch, math',self.match[0][0])
        print('end of epoch, unmatch',self.unmatch[0][0])
        
        print(len(self.match), len(train), len(self.unmatch), len(train))
        #assert len(self.match) == len(train) and len(self.unmatch) == len(train)
    
    def construct_example(self, im_A, im_B, class_idx):
        return [im_A, im_B], class_idx

In [ ]:
"""
Create a 2D score matrix of size of training data
"""

score = np.random.random_sample(size=(len(train),len(train)))

train_dl = DataLoader(
    TwoImDataset(data.train , score),
    batch_size=BS,
    shuffle=True,
    num_workers=NUM_WORKERS
)


In [ ]:
def normalize_batch(batch):
    stat_tensors = [torch.tensor(l).cuda() for l in imagenet_stats]
    return [normalize(batch[0][0], *stat_tensors), normalize(batch[0][1], *stat_tensors)], batch[1]

In [ ]:
data_bunch = ImageDataBunch(train_dl , train_dl) ##, valid_dl
data_bunch.add_tfm(normalize_batch)

In [ ]:
"""
The netowrk architecture is also inspired from Martin's notebook (part after we extract features for two image pairs)
"""
class SiameseNetwork(nn.Module):
    def __init__(self, arch=models.resnet50):
        super().__init__() 
        
        self.cnn = create_body(arch)
        self.head = nn.Linear(num_features_model(self.cnn), 1)  #
        
        self.conv1 = nn.Conv2d(1 , 32 , kernel_size= (1 , 4) , padding = 0 ,stride=1)
        self.conv2 = nn.Conv2d( 1 , 1 , kernel_size = (32 ,1 ) , padding = 0  , stride=1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, im_A, im_B):
        x1, x2 = seq(im_A, im_B).map(self.cnn).map(self.process_features)
        d1 = self.calculate_distance(x1, x2)
        d2 = (x1 + x2)
        d3 = (x1*x2)
        d4 = (x1-x2)*(x1 - x2)
        concat_layer = torch.cat([d1 ,d2,d3, d4]  ,dim = 1)
        concat_layer = concat_layer.view( - 1, 1, num_features_model(self.cnn) , 4)   ## no of channels is second dimension
        concat_layer  = F.relu(self.conv1(concat_layer))
        concat_layer = concat_layer.view(-1 ,1,32, num_features_model(self.cnn))
        concat_layer = F.relu(self.conv2(concat_layer))
        concat_layer_fn = concat_layer.view(-1 ,num_features_model(self.cnn))
        dropt = self.dropout(concat_layer_fn)
        out = self.head(dropt)
        return out
    
    def process_features(self, x): 
        y = x.reshape(*x.shape[:2], -1)
        return x.reshape(*x.shape[:2], -1).max(-1)[0]
    def calculate_distance(self, x1, x2): return (x1 - x2).abs_()

In [ ]:
learn = Learner(data_bunch, SiameseNetwork(), 
                loss_func=BCEWithLogitsFlat(), 
                wd=0.01,
                metrics=[lambda preds, targs: accuracy_thresh(preds.squeeze(), targs, sigmoid=False)])

In [ ]:
def linear_schedule(step, pars):
    "Linearly output value, end_step must greater than start_step"
    start_value = pars[0]
    end_value = pars[1]
    start_step = pars[2]
    end_step = pars[3]
    assert start_step <= end_step

    if step < start_step:
        return start_value
    elif step >= end_step:
        return end_value
    return start_value - (step - start_step) * (start_value - end_value) / (end_step - start_step)


In [ ]:
linear_decay = partial(linear_schedule, pars=(1.0, 0.05, 2, 12))
linear_decay(0)

In [ ]:
class DataloaderCallback(fastai.callbacks.tracker.TrackerCallback):
    def __init__(self, learn, schedule_pars=(1.0, 0.05, 0, 10)):
        super().__init__(learn)
        self.schedule = partial(linear_schedule, pars=schedule_pars)
        
    #def on_batch_end(self, last_loss, epoch, num_batch, **kwargs: Any) -> None:
    def on_epoch_begin(self, epoch, **kwargs: Any) -> None:
        epsilon = self.schedule(epoch)
        self.learn.data.train_ds.set_epsilon(epsilon)
        self.learn.data.train_ds.on_epoch_end()



In [ ]:
learn.split([learn.model.cnn[:6], learn.model.cnn[6:], learn.model.head])
learn.freeze_to(-1)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
cb_dl = DataloaderCallback(learn, schedule_pars=(1.0, 0.05, 0, 10))
cbs = [cb_dl]

In [ ]:
learn.fit_one_cycle(4 , 1e-3, callbacks=cbs)

In [ ]:
learn.unfreeze()

In [ ]:
max_lr = 3e-4
lrs = [max_lr/100, max_lr/10, max_lr]

In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage2_unfz')

In [ ]:
learn.load(f'{name}-stage2_unfz');

In [ ]:
learn.recorder.plot_losses()

In [ ]:
cb_dl = DataloaderCallback(learn, schedule_pars=(0.05, 0.01, 0, 10))
cbs = [cb_dl]

In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage3_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage4_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage5_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage6_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage7_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage8_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage9_unfz')


In [ ]:
learn.fit_one_cycle(10, lrs, callbacks=cbs)
learn.save(f'{name}-stage10_unfz')


In [ ]:
learn.load(f'{name}-stage9_unfz')
learn.model.cuda()

In [ ]:
new_whale_fns = set(df[df['Id']=='new_whale'].sample(frac = 1).Image.iloc[:1000])
#new_whale_fns
val_fns = set(df[df.sighting_count == 2].Image)

In [ ]:
val_fns = set(df[df.sighting_count == 2].Image)
print(len(val_fns) + len(new_whale_fns))

classes = df.Id.unique()

df = df.drop(columns = ['sighting_count'])
df.head(2)

In [ ]:

data = (
    ImageItemList
        .from_df(df, train_path, cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns.union(new_whale_fns))
        .label_from_func(lambda path: fn2label[path2fn(path)], classes=classes)
        .add_test(ImageItemList.from_folder(test_path))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=root_path)
        .normalize(imagenet_stats)
)

In [ ]:
dist_mat, val_target, _ = cal_mat(learn.model, data.valid_dl, data.valid_dl, ds_with_target1=True, ds_with_target2=True)

In [ ]:
%%time
targs = []
feats = []
learn.model.eval()
with torch.no_grad():
    for ims, ts in data.valid_dl:
        feats.append(learn.model.process_features(learn.model.cnn(ims)))  ##
        targs.append(ts)

    feats = torch.cat(feats)
    print(feats.shape)

In [ ]:
feats

In [ ]:
%%time
sims = []
learn.model.eval()
with torch.no_grad():
    for feat in feats:
        x1 = feats#.copy()
        x2 = feat.unsqueeze(0).repeat(3570 ,1)
        d1 = learn.model.calculate_distance(x1 , x2)
        d2 = (x1 + x2)
        d3 = (x1*x2)
        d4 = (x1-x2)*(x1 - x2)
        concat_layer = torch.cat([d1 ,d2,d3, d4]  ,dim = 1)
        concat_layer = concat_layer.view( - 1, 1, num_features_model(learn.model.cnn) , 4)   ## no of channels is second dimension
        concat_layer  = F.relu(learn.model.conv1(concat_layer.cuda()))
        concat_layer = concat_layer.view(-1 ,1,32, num_features_model(learn.model.cnn)  )
        concat_layer = F.relu(learn.model.conv2(concat_layer))
        concat_layer_fn = concat_layer.view(-1 ,num_features_model(learn.model.cnn) )
        #out = learn.model.head(concat_layer_fn)
        predicted_similarity = learn.model.head(concat_layer_fn).sigmoid_()  #.cuda()
        sims.append(predicted_similarity.squeeze())

    


In [ ]:
new_whale_idx = np.where(classes == 'new_whale')[0][0]
new_whale_idx

In [ ]:
top_5s = []
learn.model.eval()
with torch.no_grad():
    for i, sim in enumerate(sims):
        idxs = sim.argsort(descending=True)
        probs = sim[idxs]
        top_5 = []
        for j, p in zip(idxs, probs):
            if len(top_5) == 5: break
            if j == i: continue   
            predicted_class = data.valid_ds.y.items[j]
            """
            we dont want to predict new whale for validation data 
            """
            if predicted_class == new_whale_idx: continue
            if predicted_class not in top_5: top_5.append(predicted_class)
        top_5s.append(top_5)

    ## top 5 contains 5 best predicted classes ,w ith indices from classes dictionary

In [ ]:
top_5s[:5]

In [ ]:
"""
mapk of validation data set without having new whales in predictions. 
"""
mapk(data.valid_ds.y.items.reshape(-1,1), np.stack(top_5s), 5)

In [ ]:
# %%time
"""
trying to calcualte threshold probability for new whale, which maximises the mapk for validation data.
"""
learn.model.eval()
with torch.no_grad():
    for thresh in np.linspace(0.99, 1, 10):
        top_5s = []
        for i, sim in enumerate(sims):
            idxs = sim.argsort(descending=True)
            probs = sim[idxs]
            top_5 = []
            for j, p in zip(idxs, probs):
                if new_whale_idx not in top_5 and p < thresh and len(top_5) < 5: top_5.append(new_whale_idx)
                if len(top_5) == 5: break
                if j == i: continue
                predicted_class = data.valid_ds.y.items[j]
                if predicted_class not in top_5: top_5.append(predicted_class)
            top_5s.append(top_5)
        print(thresh, mapk(data.valid_ds.y.items.reshape(-1,1), np.stack(top_5s), 5))

In [ ]:
data = (
    ImageItemList
        .from_df(df, train_path, cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in {'69823499d.jpg'}) # in newer version of the fastai library there is .no_split that could be used here
        .label_from_func(lambda path: fn2label[path2fn(path)], classes=classes)
        .add_test(ImageItemList.from_folder(test_path))
        .transform(None, size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path=root_path)
        .normalize(imagenet_stats)
)

In [ ]:
%%time
test_feats = []
learn.model.eval()
with torch.no_grad():
    for ims, _ in data.test_dl:
        test_feats.append(learn.model.process_features(learn.model.cnn(ims)))


    test_feats = torch.cat(test_feats)


In [ ]:
%%time
train_feats = []
train_class_idxs = []
learn.model.eval()
with torch.no_grad():
    for ims, t in data.train_dl:
        train_feats.append(learn.model.process_features(learn.model.cnn(ims)))
        train_class_idxs.append(t)

    train_class_idxs = torch.cat(train_class_idxs)
    train_feats = torch.cat(train_feats)

In [ ]:
len(train_class_idxs)
len(train_feats)

In [ ]:
train_feats

In [ ]:
test_feats[0].expand(len(train_feats), 2048).shape

In [ ]:
train_feats = train_feats.cuda()
test_feats = test_feats.cuda()

In [ ]:
%%time
torch.cuda.empty_cache()
sims = []
learn.model.eval()
with torch.no_grad():
    tmp_batch_size = 1000
    for i, feat in enumerate(test_feats):
        if i % 1000 == 0:
            print(i, len(test_feats))
        #dists = learn.model.calculate_distance(train_feats, feat.unsqueeze(0).repeat(len(train_feats), 1))
        row_sim = []
        for k in range(0, len(train_feats), tmp_batch_size):
            x1 = train_feats[k:k+tmp_batch_size]
            x2 = feat.expand(x1.shape)
            d1 = learn.model.calculate_distance(x1 , x2)
            d2 = (x1 + x2)
            d3 = (x1*x2)
            d4 = (x1-x2)*(x1 - x2)
            concat_layer = torch.cat([d1 ,d2,d3, d4]  ,dim = 1)
            concat_layer = concat_layer.view( - 1, 1, num_features_model(learn.model.cnn) , 4)   ## no of channels is second dimension
            concat_layer  = F.relu(learn.model.conv1(concat_layer.cuda()))
            concat_layer = concat_layer.view(-1 ,1,32, num_features_model(learn.model.cnn)  )
            concat_layer = F.relu(learn.model.conv2(concat_layer))
            concat_layer_fn = concat_layer.view(-1 ,num_features_model(learn.model.cnn) )
            predicted_similarity = learn.model.head(concat_layer_fn).sigmoid_()  #.cuda()
            row_sim.append(predicted_similarity)
        row_sim = torch.cat(row_sim)
        sims.append(row_sim.squeeze().detach().cpu())




In [ ]:

thresh = 0.996 #0.95

top_5s = []

for sim in sims:
    idxs = sim.argsort(descending = True)
    probs = sim[idxs]
    top_5 = []
    
    
    for  i , p in zip(idxs , probs):
        if new_whale_idx not in top_5 and p < thresh  and len(top_5) < 5:
            top_5.append(new_whale_idx)
        if len(top_5) ==5: break
        #if i == new_whale_idx: continue
        predicted_class = train_class_idxs[i]
        #print(predicted_class)
        if predicted_class == new_whale_idx: continue
        if predicted_class not in top_5:
            top_5.append(predicted_class)
    top_5s.append(top_5)
    

In [ ]:
len(top_5s)

In [ ]:
top_5_classes  = []

for top_5 in top_5s:
    top_5_classes.append(' '.join([classes[t] for t in top_5]))

In [ ]:
top_5_classes

In [ ]:
sub = pd.DataFrame({'Image': [path.name for path in data.test_ds.x.items]})
sub['Id'] = top_5_classes
sub.to_csv(f'../submission/{name}.csv', index=False) 



In [ ]:
pd.read_csv(f'../submission/{name}.csv').Id.str.split().apply(lambda x: x[0] == 'new_whale').mean()

In [ ]:
#name = 'Ensembleing_resnet50_renet101_siamene_v1'
! kaggle competitions submit -c humpback-whale-identification -f ../submission/martin_siamene_network_15k_training_images.csv -m "resnet18 arch prob 1 for new whales"